# IMPORTING GZ FILE

In [258]:
import gzip

with gzip.open('./NY.gz', 'rb') as f:
    gzipFile = f.read()
    
print(gzipFile[2])

57


# GRAPH CLASS

In [259]:
class Graph():
    def __init__(self):
        print("Created Graph! \n")
        self.vertices = []
        self.edges = {}

    def addVertex(self, vertex):
        if (self.isEmpty()):
            self.vertices.append(vertex)
            self.edges.update({vertex: []})
        else:
            self.vertices.append(vertex)
            self.edges.update({vertex: []})
            # Connecting last two vertexes in graph together
            self.addEdges(self.vertices[len(
                self.vertices)-1], self.vertices[len(self.vertices)-2])

    def addEdges(self, vertex1, vertex2):
        # self.edges[vertex1].append(vertex2)
        self.edges[vertex2].append(vertex1)

    def isEmpty(self):
        return True if not len(self.vertices) else False

    def printGraph(self):
        for x in self.vertices:
          print(x.data, x.weight, [f'{city.data} weight: {city.weight}' for city in self.edges[x]])
        # x = PrettyTable()
        # x.field_names = [
        #     "Vertices (Cities)", "Edges (Cities/Distance)"]

        # try:
        #     for v in self.vertices:
        #         x.add_row([v.data, [x.getInfo() for x in self.edges[v]]])
        #     print(x)
        # except:
        #     print("Graph Vertices is not consisted of Cities from JSON file.")

# NODE CLASSES

In [260]:
# Base Vertex Class
class Vertex():
    def __init__(self, data):
        self.data = data

# Weighted Vertex
class WeightedVertex(Vertex):
    def __init__(self, data, weight):
        super().__init__(data)
        self.weight = weight

    def getInfo(self):
        return (self.data, self.weight)

# Node for BST
class Node: 
  def __init__(self, data):
    self.data = data
    self.right = None
    self.left = None
  

# BINARY TREE

In [261]:
class BinaryTree():
  def __init__(self):
    self.root = None
  
  def isEmpty(self):
    return (self.root == None)

  def getRoot(self):
    return (self.root)
  
  def add(self, node):
    if (self.isEmpty()):
      self.root = Node(node)
    else:
      self._add(node, self.root)

  def _add(self, node, parent):
    # Traversing Left
    if (node < parent.data):
      if parent.left is not None:
        self._add(node, parent.left)
      else:
        parent.left = Node(node)
    else:
      if parent.right is not None:
        self._add(node, parent.right)
      else:
        parent.right = Node(node)

  def printTree(self):
    if(self.isEmpty() is not True):
      self._printTree(self.root)

  def _printTree(self, node):
    if node is not None:
      self._printTree(node.left)
      print(str(node.data) + ' ')
      self._printTree(node.right)


In [262]:
kruskalTree = BinaryTree()

# ADD GZIP FILE TO GRAPH

In [263]:
cityGraph = Graph()
for idx, val in enumerate(gzipFile[0:100]):
    cityName = "ny city " + str(idx)
    cityVertex = WeightedVertex(cityName, val)
    cityGraph.addVertex(cityVertex)

Created Graph! 



# PRIMS ALGORITHM
---
#### - keeps track of visited
#### - greedy


In [264]:
######UTILITY FUNCTIONS########
import random

def returnWeight(obj):
  return obj["weight"]

def addRandomEdges(graph):
  for _ in range(len(graph.vertices)):
    idx1 = random.randint(0, len(graph.vertices)-1)
    idx2 = random.randint(0, len(graph.vertices)-1)
    graph.addEdges(graph.vertices[idx1], graph.vertices[idx2])
    

In [265]:
def prims(graph,start, goal, initial = True, visited = []):
  weights = []

  # base case
  if (start.data == goal.data):
    return True
  
  # change weights on start
  if (initial):
    for x in graph.vertices:
      if (start.data == x.data):
        x.weight = 0
        start.weight = 0
        initial = False

  # loop through start node edges
  for x in graph.vertices:     # x is the nodes
    if (start.data == x.data):
      for y in graph.edges[x]:    # y is the edge nodes of x 
        weights.append({"node": y,"weight" : y.weight})   #append all of the nodes and corresponding weights
      
      #Checking if they are visited already and we will pop them off the weights list
      if (len(visited) >= 1):
        for check in weights:
          for visit in visited:
            if (check["node"] == visit):
              weights.remove(check)

      # Next we visit the one with the least weight and append the node to visited
      if (len(weights) >= 1):
        weights.sort(key = returnWeight)  #sort on weight in ascending order
        visited.append(weights[0]["node"])
      else:
        visited.append(weights[0]["node"])

  for x in visited:
    if (x.data == goal.data):
      return True

  print(visited[-1].data)  

  try:
    return prims(graph, visited[-1], goal, initial, visited)    

  except IndexError:
    print("Encountered Leaf: Backtracking...")
    return prims(graph, weights[0]["node"], goal, initial, visited)
  


# KRUSKAL ALGORITHM
#### - smallest edges that dont create cycle
#### - sorts edges
#### - makes minimum spanning tree


In [266]:
def kruskal(graph, start, goal, initial=True):
  edges = []

  # base case
  if (start.data == goal.data):
    return True

  # change weights on start
  if (initial):
    for x in graph.vertices:
      if (start.data == x.data):
        x.weight = 0
        start.weight = 0
        initial = False

  # loop through vertices and sort the edges
  for x in graph.vertices:
    edges.append({"city" : x.data, "weight" : x.weight})
    edges.sort(key = returnWeight)
  
  for x in edges:
    #base case
    if (x["city"] == goal.data):
        kruskalTree.printTree()
        return True
    else:
        kruskalTree.add(x["weight"])
  




# TESTING ALGORITHMS
### Finding two nodes in graph to set as start and goal node also showing the random edges of the graph

In [267]:
addRandomEdges(cityGraph)
# cityGraph.printGraph()

#### Lets pick the first and tenth element - we have to make them a vertex class object tho

In [268]:
startNode = WeightedVertex("ny city 0", 99)
goalNode = WeightedVertex("ny city 10", 67)

#### Now lets call Prims

In [269]:
%time
print(prims(cityGraph, startNode, goalNode))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10.5 µs
ny city 1
ny city 2
ny city 3
ny city 4
ny city 5
ny city 6
ny city 28
ny city 29
ny city 0
ny city 67
ny city 68
ny city 41
ny city 42
ny city 43
ny city 37
ny city 38
ny city 39
ny city 49
True


# Lets Call Kruskal

In [270]:
%time
kruskal(cityGraph, startNode, goalNode)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs
0 
10 
10 
32 
32 
32 
32 
32 
32 
32 
32 
46 
46 
46 
47 
47 
47 
49 
57 
57 
58 
58 
65 


True